### Cargamos Ontologia y Vocabulario

In [112]:
from ontologia import *

In [7]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../Scripts')
import pandas as pd 
from ontologia import *
import ast
import difflib
#Frequency
from collections import Counter 
from itertools import repeat, chain 

#ElasticSearch
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from time import time

from numpy import (
    dot, float32 as REAL
)
import numpy as np
from elasticsearch import Elasticsearch


#Lectura de lemas 
import json
with open('./Vocabulario/diccionario-lemas.json', 'r') as f:
    diccionario_lemas = json.loads(f.read())

lemas = list(diccionario_lemas.keys())

es = Elasticsearch()
es.ping()

True

In [8]:
modeloD2V = Doc2Vec.load("./Modelos/modelo2_FINALD2V.model")

**Palabras a Buscar**

In [9]:
words = ['gestor', 'conocimiento']

#### Consulta de Palabras mas similares y en contexto

In [28]:
#Retrive Vectors
keywords = []
for word in words:
    term = {"term": {"palabra": ""}}
    term['term']['palabra'] = word
    keywords.append(term)


#Aqui Añadimos las palabras
query_vecs = {
    "query":{
        "bool":{
            "should":keywords
        }
    },
    "_source": ["palabra", "norm_vec"]
}
#Calculo del Vector
vectores = es.search(index='palabras', body=query_vecs)['hits']['hits']
norm_vecs = [vector['_source']['norm_vec'] for vector in vectores]
vector_mean = np.array(norm_vecs).mean(axis=0)
unit_vec = vector_mean / np.linalg.norm(vector_mean)

#Most similar Words
query_sims = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
       },
      "script": {
        "id": "dot_product",
        "params": {
          "query_vector": unit_vec
        }
      }
    }
  },
  "size": 20,
  "_source": ["palabra", "index"]
}
most_similar_words = es.search(index='palabras', body=query_sims)['hits']['hits']
most_similar_words = [(palabra['_source']['palabra'], palabra['_score']) for palabra in most_similar_words]
most_similar_words

[('gestor', 0.81737363),
 ('conocimiento', 0.8173736),
 ('debilmente', 0.77090985),
 ('umayux', 0.74467164),
 ('sgbd', 0.7438246),
 ('sparql', 0.7421229),
 ('rdf', 0.71533084),
 ('postgresql', 0.7092361),
 ('ontologias', 0.707866),
 ('fundamentandose', 0.69965184),
 ('solicitada', 0.690233),
 ('maskana', 0.6835666),
 ('motor', 0.6802874),
 ('transformarla', 0.677389),
 ('facilite', 0.66182),
 ('adquieran', 0.66137993),
 ('buscador', 0.6595822),
 ('digitalizados', 0.6512728),
 ('flock', 0.6399753),
 ('robusto', 0.6398908)]

#### Consulta en Ontologia

In [58]:
words_ontologia=[word[0] for word in most_similar_words]

#Palabras en plural o muy similares
more_words=[]
for word in words:
    additional = difflib.get_close_matches(word, diccionario_lemas.keys(), n=5, cutoff=0.90)
    more_words.extend(additional[1:])
    
#Añadir palabras finales
words_ontologia.extend(more_words)
# Recuperacion con OR2

words_instances=[]

for word in words_ontologia:
    instance = ontologia.search(descripcion_palabra=word)
    words_instances.extend(instance)

#print(words_instances)
docs = []
for word in words_instances:
    for doc in word.palabra_describe_pi:
        docs.append(doc)
        

# sorting on bais of frequency of elements 
# result_ontologia = list(chain(i for i, c in Counter(id_docs).most_common()))
result_ontologia = list(chain(i for i, c in Counter(docs).most_common())) 
id_ontologia = [id_pi.get_id_proyecto_investigacion()[0] for id_pi in result_ontologia]

  
# printing final result 
# print("final list", str(result), "Cantidad:",len(result)) 
for index, doc in enumerate(result_ontologia[:5]):
    title = doc.get_titulo_proyecto_investigacion()
    id_pi = doc.get_id_proyecto_investigacion()
    print(f'{id_pi[0]} - {title[0]}')

422 - MOTOR DE BÚSQUEDA INTELIGENTE DE INFORMES DE INVESTIGACIÓN BASADO EN RECURSOS SEMÁNTICOS PARA EL SISTEMA DE INVESTIGACIONES DE LA UNIVERSIDAD DE NARIÑO
443 - UMAYUX: UN MODELO DE SOFTWARE DE GESTION DE CONOCIMIENTO SOPORTADO EN UNA ONTOLOGIA DINAMICA DEBILMENTE ACOPLADO CON UN GESTOR DE BASE DE DATOS PARA LA UNIVERSIDAD DE NARIÑO
320 - APROXIMACIÓN AL PENSAMIENTO JURÍDICO NARIÑENSE DESDE LAS FIGURAS DE JOSÉ RAFAEL SAÑUDO Y ERNESTO VELA ANGULO.
438 - SITAPP: UNA APLICACIÓN INTELIGENTE PARA DISPOSITIVOS MÓVILES DEL SISTEMA DE RUTAS DE TRANSPORTE URBANO DEL MUNICIPIO DE PASTO
9 - Análisis de factores de aprendizaje en seguridad alimentaria con comunidades rurales en el municipio de Pasto, departamento de Nariño.


#### Consulta de Documentos mas similares y en contexto

In [101]:
#Cantidad de Documentos 
limit_docs = len(id_ontologia)

infer_vec = [modeloD2V.infer_vector([' '.join(words)])]
vec_mean_doc = np.array(infer_vec).mean(axis=0)
unit_vec_doc = vec_mean_doc / np.linalg.norm(vec_mean_doc)

query_sims_docs = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
       },
      "script": {
        "id": "dot_product",
        "params": {
          "query_vector": unit_vec_doc
        }
      }
    }
  },
  "size": limit_docs,
  "_source": ["tag"]
}

most_similar_docs = es.search(index='documentos', body=query_sims_docs)['hits']['hits']
most_similar_docs = [(doc['_source']['tag'], doc['_score']) for doc in most_similar_docs]
id_D2V = [id_pi[0] for id_pi in most_similar_docs]
most_similar_docs

[(25, 0.1202511),
 (334, 0.10678555),
 (23, 0.09366075),
 (125, 0.090628184),
 (152, 0.076959595),
 (310, 0.07568531),
 (364, 0.07025142),
 (416, 0.06460762),
 (94, 0.059376076),
 (357, 0.05704278),
 (169, 0.055895753),
 (423, 0.05434161),
 (115, 0.052079488),
 (13, 0.051038302),
 (395, 0.04928819),
 (350, 0.045928232),
 (419, 0.04502898),
 (379, 0.0443209),
 (390, 0.043766443),
 (146, 0.042731296),
 (259, 0.042184554),
 (305, 0.04162675),
 (312, 0.0411557),
 (159, 0.03911487),
 (118, 0.038426507),
 (210, 0.037018582),
 (365, 0.03626153),
 (84, 0.034900334),
 (388, 0.034640018),
 (88, 0.034402534),
 (313, 0.034081623),
 (281, 0.03397306),
 (77, 0.033847112),
 (271, 0.033398643),
 (241, 0.032675475),
 (164, 0.03265088),
 (50, 0.03222625),
 (430, 0.032062307),
 (27, 0.030357718),
 (209, 0.030158749),
 (344, 0.029997014),
 (341, 0.02998929),
 (38, 0.02983044),
 (333, 0.029618734),
 (335, 0.029407991),
 (51, 0.029326908),
 (285, 0.028768122),
 (126, 0.028627725),
 (399, 0.028609306),
 (394

In [103]:
print(len(id_ontologia))
print(len(id_D2V))

165
165


## Comparacion de Resultados y Presentación Final

In [102]:
print(id_ontologia)
print(id_D2V)

[422, 443, 320, 438, 9, 34, 62, 63, 64, 190, 193, 204, 230, 231, 247, 248, 292, 293, 304, 311, 312, 337, 352, 375, 431, 433, 444, 1, 38, 281, 3, 4, 6, 7, 8, 10, 13, 14, 21, 24, 25, 28, 30, 39, 48, 49, 53, 55, 58, 65, 68, 73, 75, 77, 80, 83, 91, 94, 98, 103, 106, 107, 111, 117, 119, 122, 126, 127, 131, 143, 154, 157, 159, 160, 163, 168, 170, 177, 180, 188, 191, 196, 198, 201, 202, 205, 212, 214, 219, 220, 223, 224, 229, 232, 238, 251, 261, 263, 266, 271, 272, 276, 280, 282, 284, 286, 288, 291, 305, 308, 310, 321, 329, 331, 332, 334, 336, 338, 343, 345, 346, 349, 350, 360, 368, 371, 376, 377, 380, 382, 394, 405, 407, 409, 416, 417, 430, 441, 445, 448, 435, 104, 173, 197, 235, 255, 324, 395, 399, 400, 403, 12, 18, 47, 56, 71, 203, 218, 226, 233, 243, 245, 410, 419, 451]
[25, 334, 23, 125, 152, 310, 364, 416, 94, 357, 169, 423, 115, 13, 395, 350, 419, 379, 390, 146, 259, 305, 312, 159, 118, 210, 365, 84, 388, 88, 313, 281, 77, 271, 241, 164, 50, 430, 27, 209, 344, 341, 38, 333, 335, 51, 28

In [114]:
final_search = []
for onto, d2v in zip(id_ontologia, id_D2V):
    if onto == d2v:
        final_search.append(onto)
        continue
    else:
        if (onto not in final_search) and (d2v not in final_search):
            final_search.append(onto)
            final_search.append(d2v)
final_search

[422,
 25,
 443,
 334,
 320,
 23,
 438,
 125,
 9,
 152,
 34,
 310,
 62,
 364,
 63,
 416,
 64,
 94,
 190,
 357,
 193,
 169,
 204,
 423,
 230,
 115,
 231,
 13,
 247,
 395,
 248,
 350,
 292,
 419,
 293,
 379,
 304,
 390,
 311,
 146,
 312,
 259,
 337,
 305,
 375,
 159,
 431,
 118,
 433,
 210,
 444,
 365,
 1,
 84,
 38,
 388,
 281,
 88,
 3,
 313,
 6,
 77,
 7,
 271,
 8,
 241,
 10,
 164,
 14,
 430,
 21,
 27,
 24,
 209,
 28,
 341,
 39,
 333,
 48,
 335,
 49,
 51,
 53,
 285,
 55,
 126,
 58,
 399,
 65,
 394,
 68,
 361,
 73,
 137,
 75,
 187,
 80,
 235,
 83,
 411,
 98,
 133,
 103,
 366,
 106,
 26,
 107,
 314,
 111,
 437,
 117,
 385,
 119,
 300,
 122,
 392,
 127,
 346,
 131,
 176,
 143,
 258,
 157,
 393,
 160,
 440,
 163,
 12,
 168,
 97,
 170,
 42,
 177,
 243,
 180,
 81,
 188,
 428,
 196,
 331,
 198,
 340,
 202,
 414,
 205,
 79,
 220,
 110,
 229,
 144,
 232,
 200,
 238,
 244,
 251,
 308,
 272,
 212,
 276,
 108,
 282,
 92,
 284,
 301,
 286,
 124,
 288,
 90,
 291,
 386,
 321,
 234,
 329,
 134,
 336,
 2

In [115]:
type(ontologia)

owlready2.namespace.Ontology

In [121]:
final_result = [ontologia.search(id_proyecto_investigacion=id_pi)[0] for id_pi in search]
for pi in final_result:
    print(pi.get_titulo_proyecto_investigacion())


['MOTOR DE BÚSQUEDA INTELIGENTE DE INFORMES DE INVESTIGACIÓN BASADO EN RECURSOS SEMÁNTICOS PARA EL SISTEMA DE INVESTIGACIONES DE LA UNIVERSIDAD DE NARIÑO']
['APLICACIÓN DE UNA METODOLOGÍA de TRABAJO INTERDISCIPLINARIA, CON ENFOQUE DE ACCIÓN SIN DAÑO EN EL RELACIONAMIENTO ACADEMIA - COMUNIDAD PARA EL DESARROLLO LOCAL, EN LAS MICROCUENCAS DE MIRAFLORES MUNICIPIO DE TANGUA Y MIJITAYO MUNICIPIO DE PASTO, DEPARTAMENTO DE NARIÑO.']
['UMAYUX: UN MODELO DE SOFTWARE DE GESTION DE CONOCIMIENTO SOPORTADO EN UNA ONTOLOGIA DINAMICA DEBILMENTE ACOPLADO CON UN GESTOR DE BASE DE DATOS PARA LA UNIVERSIDAD DE NARIÑO']
['COMUNICACIÓN EN LA MOVILIDAD URBANA: nuevas perspectivas desde el diseño industrial (modalidad, creacion artistica y cultural)']
['APROXIMACIÓN AL PENSAMIENTO JURÍDICO NARIÑENSE DESDE LAS FIGURAS DE JOSÉ RAFAEL SAÑUDO Y ERNESTO VELA ANGULO.']
['APLICACIÓN DE LA TECNOLOGÍA DE EMPAQUE BAJO ATMOSFERAS MODIFICADAS MAP, EN LA CONSERVACIÓN Y VIDA ÚTIL DE DOS VARIEDADES DE ARVEJA (PISUM SATIVUM

In [113]:
type(ontologia)

owlready2.namespace.Ontology